# 3. Limpieza y Transformación

-----

In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = 36

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from category_encoders import WOEEncoder      
import plotly.graph_objects as go                  

In [2]:
# Importamos dataset:

hotel = pd.read_csv('../src:data/src:data:raw/city_hotel.csv', index_col=0 )
print("Nº de reservas:", hotel.shape[0], "\nFeatures:", hotel.shape[1])

hotel.head()

Nº de reservas: 79330 
Features: 31


,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,0,6,2015,July,27,1,0,2,1,0.0,0,HB,PRT,Offline TA/TO,TA/TO,0,0,0,A,A,0,No Deposit,6.0,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-03
1,1,88,2015,July,27,1,0,4,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9.0,NaN,0,Transient,76.5,0,1,Canceled,2015-07-01
2,1,65,2015,July,27,1,0,4,1,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9.0,NaN,0,Transient,68.0,0,1,Canceled,2015-04-30
3,1,92,2015,July,27,1,2,4,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9.0,NaN,0,Transient,76.5,0,2,Canceled,2015-06-23
4,1,100,2015,July,27,2,0,2,2,0.0,0,BB,PRT,Online TA,TA/TO,0,0,0,A,A,0,No Deposit,9.0,NaN,0,Transient,76.5,0,1,Canceled,2015-04-02


------

| **Resumen Variables Numéricas**|
| :---: |

!! No tenemos correlación lineal con el target

| **Variable** | **Card (%)** |**Media** | **Mediana** | **Outliers (%)** | **Distr. Normal**  | **Shapiro** | **Pearson**| **Phik** |
| :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- | :--- |
| dias_modificacion_llegada | 0.48 | 38.94 | 4 | 14.9 | No | 0 | 0.57 | 0.78 | 
| lead_time | 0.57 | 109.7 | 74 | 3.25 |  No | 0 | 0.31 | 0.39 |
| total_of_special_requests | 0.01 | 0.54 | 0 | 2.28 | No |0 | -0.29 | 0.46 | 
| stays_in_week_nights | 0.04 | 2.18 | 2 | 0.91 | No | 0 | 0.049 | 0.051 |
| required_car_parking_spaces | 0.01 | 0.024 | 0 | 2.43 | No | 0 | -0.13 | 0.2 |
| days_in_waiting_list | 0.14 | 3.22 | 0 | 4.34 | No | 0 | 0.061 | 0.076 | 
| booking_changes | 0.03 | 0.18 | 0 | 12.94 | No | 0  | -0.15 | 0.059 |
| previous_cancellations | 0.01 | 0.079 | 0 | 6.79 | No | 0 | 0.17 | 0.032 |
| previous_bookings_not_canceled | 0.09 | 0.13 | 0 | 2 | No | 0 | -0.053 | 0.062 | 
| total_n_noches | 0.05 | 2.98 | 3 | 1.57 | No | 0 | 0.035 | 0.039 |
| stays_in_weekend_nights | 0.02 | 0.79 | 1 | 0.1 | No | 0 | -0.0073 | 0.038 |
| babies | 0.01 | 0.0049 | 0 | 0.47 | No | 0 | -0.03 | 0.027 |
| children | 0.01 | 0.09 | 0 | 6.44 | No | 0 | -0.027 | 0.054 | 
| adults | 0.01 | 1.85 | 2 | 0.01 | No | 0 | 0.053 | 0.07 |
| adr | 6.81 | 105.3 | 99.9 | 4.27 | No | 0 | -0.012 | 0 |

| **Pasos Limpieza**|
| :---: |

**Missings**:

- Eliminamos **company** porque tiene un 95.5% de nulos

**Outliers**:

- **dias_modificacion_llegada**: todos los outliers son reservas canceladas, interpretamos que a mayor antelación, mayor posibilidad de cancelación, por lo que no eliminamos la variable.
- **lead_time**: el 70% de los outliers son reservas que se cancelan. Entendemos que a mayor antelación en la reserva, mayor posibilidad de cancelación por lo que no los eliminamos.
- **days_in_waiting_list**: solo el 4.36% de las reservas entran en lista de espera y el 41.73% de las reservas que entran en lista de espera cancelan. Cualquier valor diferente a 0 es un outlier en esta columna, por lo que eliminarnos sería como quitar la columna. 
- **previous_cancellations**: la mayoría de outliers están entre 1 y  2 cancelaciones previas, tienen sentido y quitar estos datos sería como eliminar la columna. De 111 reservas con más de 3 cancelaciones previas, 98 también tienen reservas previas no canceladas (huéspedes que no repiten) --> esto nos indica huéspedes muy habituales, por lo que no los eliminamos.
- **previous_bookings_not_cancelled**: interpretamos como huéspedes habituales.
- **total_n_noches**: El 54% cancelan. Las mantenemos porque aunque no es lo más común, sí tienen sentido reservas largas de hotel.
- **stay_in_week_nights**: El 58.6% de las estancias largas, cancelan. Mismo razonamiento que total_n_noches.
- **stay_in_weekend_nights**: Mismo razonamiento que total_n_noches.
- ***adr**: mantenemos outliers ya que esta columna se define por su variabilidad en los precios según día. Eliminaremos 1 reserva con precio de 5400€ ya que el precio más alto antes de éste es de 510€ y probablemente 5400€ sea una errata.
- **booking_changes**: no eliminamos outliers ya que esta columna todos los valores distintos de 0 son outliers.
- **total_of_special_requests**: lo mismo que booking_changes.
- **required_car_parking_spaces**: lo mismo que booking_changes.
- **adults**: el valor máximo de número de adultos es 4 (30 reservas con este número). Aunque no es lo más común, tampoco podría clasificarse de atípico reservas con 3 o 4 adultos. Los mantenemos.
- **niños**: aunque hay un 6.44% de outliers porque lo más común en el hotel son 0 niños, no los quitamos ya que tiene sentido de 0 - 3 niños.
- **bebés**: eliminaremos las reservas con + 2 bebés. Estas reservas tienen 9 y 10 bebés y es muy atípico (son solo 2 filas).

**Variables que eliminamos**:

- **dias_modificacion_llegada**: la creamos para el análisis, pero tiene una correlación linea de 1 con nuestro target -no la eliminamos, simplemente no la crearemos.
- **total_n_noches**: la creamos para el análisis pero no la crearemos para el modelo (muy relacionada con stay_week_nights) y nos da la misma información que incluir las variables: stay_week_nights y stay_weekend_nights.


----

| **Resumen Variables Categóricas**|
| :---: |

!! Pocas features con variabilidad en el peso de las categorías

| **Variable** | **Descripción** | **CARD (%)** | **N Categorías** |  **Moda** | **Peso Moda** |**Phik**| 
| :--- | :--- | :--- | :--- | :--- | :--- | :--- | 
| agent | ID del agente o agencia | 0.28 | 333 | 9 | 26.77 | 0.091 |
| company | ID compañía o entidad| 0.29 | 352 |40 | 0.78 | - | 
| country | País procedencia huésped | 0.15 | 177 | PRT | 40.7 |  0.5 | 
| market_segment | Modo de reserva | 0.01 | 8 | Online TA | 47.4 | 0.39 | 
| assigned_room_type | Tipo de Hab Asignada | 0.01 | 12 | A | 62.03 |  0.18 |
| reserved_room_type | Tipo de Hab Reservada | 0.01 | 10 | A | 72.03 |  0.1 | 
| reservation_status | Último estado de la Reserva | 0 | 3 | Check-Out | 62.96 |  1 | 
| customer_type | Tipo de Huésped | 0 | 4 |Transient | 75.06 | 0.23 | 
| meal | Paquete de Comidas Reservado | 0 | 5 | BB | 77.32 | 0.069 | 
| distribution_channel | Canal de Distribución | 0 | 5 | TA/TO | 81.98 |  0.14 | 
| deposit_type | Indica Tipo de Depósito | 0 | 3 | No Deposit | 87.65 | 0.32 | 

(Ordenadas por % de cardinalidad)

| **Pasos Limpieza**|
| :---: |

**Missings**:

- **children** (0.01%): sustituímos por 0 porque entendemos que en las 4 reservas con missings no había niños.
- **country** (0.03%): sustituímos por la moda (PRT).
- **agent** (10.25%): sustituímos por 0 para representar que ningún agente o agencia realizó la reserva.

**Variables que eliminamos**:

- **company**: 95.35% de missings.
- **reservation_status**: variable equivalente al target.
- **reserved_room_type**: muy correlacionada con assigned_room_type, elegimos assigned_room_type porque nos da un valor -en la matriz phik- un poco más alto de correlación lineal que reserved_room_type (0.18 - 0.1).
- **agent**: tras realizar un primer análisis, nos dimos cuenta que el 96.5% de todas las cancelaciones del dataset están agrupadas dentro de la categoría 9 --> no sabemos por qué, por lo que para no contaminar el análisis eliminamos esta columna.
- **deposit_type**: al realizar un primer análisis vimos que todas las reservas que estaban en la categoría Non Refund, eran reservas canceladas. No sabemos por qué y, como no tiene sentido, eliminamos esta columna para evitar contaminar el análisis.

**Otros**:

- **market_segment**: eliminamos las filas con categoría 'Undefined' (2).
- **distribution_channel**: eliminamos las filas con categoría 'Undefined (4).

---

| **Resumen Variables Numéricas - Categóricas**|
| :---: |

| **Variable** | **CARD (%)** | **N Categorías** |  **Moda** | **Peso Moda** |**Phik**| 
| :--- |  :--- | :--- | :--- | :--- |:--- | 
| is_repeated_guest | 0 | 2 | 0: Huésped Nuevo | 96.8 | 0.1 |

| **Pasos Limpieza**|
| :---: |

**Missings**:

- No hay

**Variables que eliminamos**:

- Ninguna

---

| **Resumen Variables Fechas**|
| :---: |

| **Variable** |  **Valores** |**Comentarios** | **Phik**|
| :--- |  :--- | :--- | :--- | 
| reservation_status_date |  - | Mediana días antelación cancelación: 54 | 0.66| 
| arrival_date | - | - | 0.4 |
| arrival_date_month | Ene - Dic | - | 0.669 | 
| arrival_date_week_number |  1 - 53 | - | 0.065 |
| arrival_date_year | 2015-2016-2017 | 1/07/15 al 31/08/17 --> año completo: 2016 | 0.045 | 
| arrival_date_day_of_month | 1 - 31 | - | 0.051 | 

| **Pasos Limpieza**|
| :---: |

**Missings**:

- Ninguno

**Variables que eliminamos**:

- **arrival_date**: variable creada para el análisis, sacada sumando las variables: arrival_date_day_of_month, arrival_date_month y arrival_date_year. No la incluiremos porque utilizaremos las ya dadas (arrival_date_day_of_month, arrival_date_month - al no tener formato fecha es más fácil para incluir en el modelo).
- **arrival_date_year**: no tenemos los 3 años completos, por lo que esta variable no ayuda mucho a marcar diferencias.
- **arrival_date_week_number**: 0.98 de correlación lineal con arrival_date_month.
- **reservation_status_date**: 0.96 de correlación lineal con arrival_date_month y 0.84 con dias_modificacion_llegada y lead_time. 

**Otros**:

- **market_segment**: eliminamos las filas con categoría 'Undefined' (2).
- **distribution_channel**: eliminamos las filas con categoría 'Undefined (4).

---

| **Target**| **Tipo**| **Valores**|
| :--- | :--- | :--- |
| 'is_canceled' | Binario | 1: Reserva Cancelada; 0: Reserva No Cancelada

---

| **Errores a eliminar**| 
| :---: |

- 4963 reservas que no tienen ni noches entre semana ni en fin de semana (solo 1480 fueron canceladas)

- 97 reservas sin adulto, bebés o niños (solo 23 canceladas)

- 3 reservas con bebés pero sin adultos

- 2 reservas con más de 8 bebés

- 313 reservas que tienen misma fecha de última modificación y fecha de llegada, pero como estado está Check-Out --> la mayoría se eliminan en los 4 puntos anteriores, quedarían 5.


---

In [3]:
# 1 - Eliminaciones previas a dividir en train y test

# 1.1.  Errores en el dataset

#       Reservas con 0 noches
hotel.drop(hotel[(hotel['stays_in_week_nights'] == 0) & (hotel['stays_in_week_nights'] == 0)].index, inplace=True)

#       Reservas sin adultos, bebés o niños
hotel.drop(hotel[(hotel['adults'] == 0) & (hotel['children'] == 0) & (hotel['babies'] == 0)].index, inplace = True)

#       Reservas con bebés pero sin adultos
hotel.drop(hotel[(hotel['adults'] == 0) & (hotel['babies'] > 0)].index, inplace = True)

#       Reservas con + de 8 bebés
hotel.drop(hotel[hotel['babies'] > 8].index, inplace=True)

#       Misma fecha de última modificación y fecha de llegada, pero como estado está Check-Out 
hotel.drop(hotel[(pd.to_datetime(hotel['reservation_status_date']) == \
        (pd.to_datetime(hotel['arrival_date_year'].map(str) +  "-" + hotel['arrival_date_month'].map(str) + "-" + hotel['arrival_date_day_of_month'].map(str))))\
             & (hotel['reservation_status'] == 'Check-Out')].index, inplace=True)

# 1.2.  Outliers

#       Outlier columna 'adr'
hotel.drop(hotel[hotel['adr'] > 5000].index, inplace = True)

#       market segment undefined 
hotel.drop(hotel[hotel['market_segment'] == 'Undefined'].index, inplace=True)

#       distribution channel undefined 
hotel.drop(hotel[hotel['distribution_channel'] == 'Undefined'].index, inplace=True)


In [4]:
# 2 - Dividimos en train y test:

X = hotel.drop(['is_canceled'], axis = 1)
y = hotel['is_canceled']

X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size = 0.2,
                                                    random_state = 42)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)  

# 2.1 - Vemos balanceados de nuestro target --> 0.57 No Cancelan, 0.42 Cancelan 
#                                               Balanceo dataset hotel (sin quitar errores) --> si: 0.42, no: 0.58 (ver_ 2_EDA).

# Función para ver balanceado:

def balanceado_target(series):
    balanceado = series.value_counts()
    return round(balanceado[0] / series.shape[0], 2), 1 - balanceado[0] / series.shape[0]

print("Balanceado y_train:", balanceado_target(y_train))
print("Balanceado y_test:", balanceado_target(y_test)) 

(59404, 30) (59404,)
(14852, 30) (14852,)
Balanceado y_train: (0.57, 0.42652009965658877)
Balanceado y_test: (0.58, 0.4220307029356316)


In [5]:
# 3 - Limpieza X_train, X_test

# 3.1. Eliminamos columnas que no usaremos: 6

var_eliminar = ['company', 'reservation_status', 'reserved_room_type', 'reservation_status_date',
                'arrival_date_year', 'arrival_date_week_number', 'agent', 'deposit_type']

for column in X_train[var_eliminar]:
    del X_train[column]

for column in X_test[var_eliminar]:
    del X_test[column]    

X_train.shape, X_test.shape      

((59404, 22), (14852, 22))

In [6]:
# 3.2. Tratamos Missings:

#     country --> sustituimos por la moda de la categoría
print("Moda X_train['country']:", X_test['country'].value_counts().index[0])
X_train['country'].fillna('PRT', inplace = True) 
X_test['country'].fillna('PRT', inplace = True) 

#     children --> 0 # poner la mediana
X_train['children'].fillna(0, inplace = True)
X_test['children'].fillna(0, inplace = True)

Moda X_train['country']: PRT


In [7]:
# Nos aseguramos que no hay nulos y vemos tipos
#X_train.info()
#X_test.info()

In [7]:
# 2.3. Tipos: Nos aseguramos del tipo de todas las variables

#     Pasamos 'arrival_date_month' a datetime, y después a int, para pasar los meses de nombre ('enero') a número(1)
X_train['arrival_date_month'] = pd.to_datetime(X_train['arrival_date_month'], format='%B').dt.month.astype(int)
X_test['arrival_date_month'] = pd.to_datetime(X_test['arrival_date_month'], format='%B').dt.month.astype(int)

In [8]:
#       'arrival_date_month', 'arrival_date_day_of_month' y 'is_repeated_guest están en int, pero son categóricas
#       Código de nb ML Guide

tipos_vars = {str: ['meal', 'country', 'market_segment', 'distribution_channel', 'assigned_room_type', 'customer_type'], 
            int: ['lead_time', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'is_repeated_guest',
                    'previous_cancellations', 'previous_bookings_not_canceled', 'booking_changes', 'required_car_parking_spaces', 'total_of_special_requests',
                    'days_in_waiting_list', 'arrival_date_day_of_month', 'children', 'babies'],
                float: ['adr']    
}

for key, value in tipos_vars.items():
      for i in value:
            X_train[i] = X_train[i].astype(key, errors = 'ignore')
            X_test[i] = X_test[i].astype(key, errors = 'ignore')

# X_train.info()
# X_test.info()            

In [9]:
# 2.5. Encoding Variables Categóricas: Entendemos todas las variables como nominales 
#       las variables de mes y dias las dejamos como están

# 2.5.1. Dummies --> lo aplicamos a variables con 4 o menos categorias
#                    todas las variables contienen categorías predefinidas -opciones cerradas-, 
#                    por lo que train contiene todas las categorías posibles de test                   

#      meal: 4
#      deposit_type: 3
#      customer_type: 4
#      distribution_channel: 4
#      No aplicamos dummy a 'is repeated guest' porque ya está clasificada con 0: No repite, 1: repite.

X_train = pd.get_dummies(X_train, columns=['meal', 'customer_type', 'market_segment', 'distribution_channel' ], 
                            prefix = ['meal', 'customer', 'market_segment', 'distribution_channel'], drop_first = True)
X_train.head()

,lead_time,arrival_date_month,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,country,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,assigned_room_type,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,meal_FB,meal_HB,meal_SC,customer_Group,customer_Transient,customer_Transient-Party,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO
32205,464,7,25,0,1,2,0,0,PRT,0,0,0,A,0,0,90.0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1
18699,34,10,15,2,3,2,0,0,FRA,0,0,0,D,0,0,166.8,0,2,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
53232,130,7,17,2,1,2,0,0,IRL,0,0,0,A,0,0,105.3,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
77553,242,8,5,1,1,2,2,0,GBR,0,0,0,F,0,0,202.5,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
28820,30,5,22,1,1,1,0,0,PRT,0,0,0,A,0,0,144.0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0


In [10]:
X_test = pd.get_dummies(X_test, columns=['meal', 'customer_type', 'market_segment', 'distribution_channel' ], 
                            prefix = ['meal','customer', 'market_segment', 'distribution_channel'], drop_first = True)
X_test.head()

,lead_time,arrival_date_month,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,country,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,assigned_room_type,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,meal_FB,meal_HB,meal_SC,customer_Group,customer_Transient,customer_Transient-Party,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO
36434,414,12,5,2,1,2,0,0,PRT,0,1,0,A,0,0,62.00,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
58903,0,10,6,0,1,2,2,0,ISR,0,0,0,F,0,0,216.00,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
49997,106,5,29,2,1,2,0,0,DEU,0,0,0,A,0,0,80.75,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1
51498,35,6,21,0,2,1,0,0,PRT,0,0,0,B,1,0,107.10,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
37112,344,9,26,0,1,1,0,0,PRT,0,1,0,A,1,0,170.00,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1


In [12]:
# WOE --> mide la 'fuerza'? de una técnica de agrupamiento para separar bueno y malo
#           En qué medida la evidencia apoya o socava una hipótesis
#           Ej. loan -->
# WOE es 0 si P(Bueno) / P(Malo) = 1, es decir, el outcome es aleatorio para ese grupo
# Si mayor % de malo que de bueno, las probabilidades serán < 1, entonces WOE será < 0
# al revés, P(Buenos) > Malors, Woe es mayor que 0

# parámetro 'handle_unknown': default 0 
# parámetro 'regularization': si = 1 se evita división entre 0 

# WOE(x) es positivo o negativo en función de si x es más representativa de positivos o negativos

In [11]:
# 2.5.2. Aplicamos el WOE a las variables 'country', 'agent' y 'assigned_room_type'

woe = WOEEncoder(cols=['country', 'assigned_room_type'], random_state=42, regularization=1) # regularization = 1 para evitar divisiones entre 

#       Entrenamos y transformamos X_train
X_train_woe = X_train[['country', 'assigned_room_type']] # las columnas que quiero transformar
y_train_woe = y_train # nuestro target
X_train[['country', 'assigned_room_type']] = woe.fit_transform(X_train_woe, y_train_woe)

#       Transformamos X_test
X_test_woe = X_test[['country', 'assigned_room_type']] 
y_test_woe = y_test 
X_test[['country', 'assigned_room_type']] = woe.transform(X_test_woe, y_test_woe)

In [12]:
print(X_train.shape)
X_train.head()

(59404, 33)


,lead_time,arrival_date_month,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,country,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,assigned_room_type,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,meal_FB,meal_HB,meal_SC,customer_Group,customer_Transient,customer_Transient-Party,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO
32205,464,7,25,0,1,2,0,0,0.990232,0,0,0,0.221831,0,0,90.0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1
18699,34,10,15,2,3,2,0,0,-1.065651,0,0,0,-0.597884,0,0,166.8,0,2,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
53232,130,7,17,2,1,2,0,0,-0.416522,0,0,0,0.221831,0,0,105.3,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
77553,242,8,5,1,1,2,2,0,-0.565356,0,0,0,-0.218601,0,0,202.5,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
28820,30,5,22,1,1,1,0,0,0.990232,0,0,0,0.221831,0,0,144.0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0


-------

| **Feature Importance**|
| :---: |

Select KBest

-----> Esto al final no lo usé en el modelo 

In [13]:
X_train.shape

(59404, 33)

In [14]:
X_test.shape

(14852, 33)

In [15]:
# Vemos qué variables son más importantes --> selectKbest

from sklearn.feature_selection import SelectKBest
sel = SelectKBest(k=33) # para ordenar todas las columnas por importancia

X_train_kbest = sel.fit_transform(X_train, y_train)
print("X_train.shape:", X_train.shape)
print("X_train_kbest.shape:",X_train_kbest.shape)


# Código de https://rotadev.com/the-easiest-way-for-getting-feature-names-after-running-selectkbest-in-scikit-learn-dev/

names = X_train.columns.values[sel.get_support()]
scores = sel.scores_[sel.get_support()]

names_scores = list(zip(names, scores))

importance_df = pd.DataFrame(data = names_scores, columns=['Feature_kbest', 'Score_kbest'])
importance_df = importance_df.sort_values(['Score_kbest'], ascending=False).reset_index(drop=True)
importance_df

X_train.shape: (59404, 33)
X_train_kbest.shape: (59404, 33)


,Feature_kbest,Score_kbest
0,country,12454.663774
1,lead_time,6206.308062
2,total_of_special_requests,5785.575221
3,market_segment_Groups,3987.019960
4,assigned_room_type,2027.503678
5,distribution_channel_TA/TO,1778.042939
6,previous_cancellations,1727.523129
7,customer_Transient-Party,1406.665321
8,booking_changes,1400.161106
9,distribution_channel_Direct,1263.751659


In [16]:
importance_df[:10]

,Feature_kbest,Score_kbest
0,country,12454.663774
1,lead_time,6206.308062
2,total_of_special_requests,5785.575221
3,market_segment_Groups,3987.019960
4,assigned_room_type,2027.503678
5,distribution_channel_TA/TO,1778.042939
6,previous_cancellations,1727.523129
7,customer_Transient-Party,1406.665321
8,booking_changes,1400.161106
9,distribution_channel_Direct,1263.751659


In [17]:
# GRÁFICO PRINCIPALES VARIABLES

fig = go.Figure()
fig.add_trace(go.Bar(name='Feature Importance Random Forest', y=importance_df['Score_kbest'][:10], 
                    x=importance_df['Feature_kbest'][:10], marker_color = '#4620ff'))

fig.update_layout(barmode='group', uniformtext_minsize=12, uniformtext_mode='hide', font_family = 'Arial', title_font_family='Arial', 
                        plot_bgcolor='#efefef', paper_bgcolor='#efefef',showlegend=False)

fig.update_yaxes(visible = True)

fig.show()

----

In [20]:
# 3. Guardamos los datasets

#X_train.to_csv('../src:data/src:data:processed/X_train.csv', encoding='utf-8')

In [18]:
print(y_train.shape)
y_train.head()

(59404,)


32205    1
18699    1
53232    0
77553    0
28820    1
Name: is_canceled, dtype: int64

In [22]:
#y_train.to_csv('../src:data/src:data:processed/y_train.csv', encoding='utf-8')

In [19]:
print(X_test.shape)
X_test.head()

(14852, 33)


,lead_time,arrival_date_month,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,country,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,assigned_room_type,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests,meal_FB,meal_HB,meal_SC,customer_Group,customer_Transient,customer_Transient-Party,market_segment_Complementary,market_segment_Corporate,market_segment_Direct,market_segment_Groups,market_segment_Offline TA/TO,market_segment_Online TA,distribution_channel_Direct,distribution_channel_GDS,distribution_channel_TA/TO
36434,414,12,5,2,1,2,0,0,0.990232,0,1,0,0.221831,0,0,62.00,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
58903,0,10,6,0,1,2,2,0,-0.882612,0,0,0,-0.218601,0,0,216.00,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
49997,106,5,29,2,1,2,0,0,-1.239021,0,0,0,0.221831,0,0,80.75,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1
51498,35,6,21,0,2,1,0,0,0.990232,0,0,0,-0.778220,1,0,107.10,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
37112,344,9,26,0,1,1,0,0,0.990232,0,1,0,0.221831,1,0,170.00,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1


In [24]:
#X_test.to_csv('../src:data/src:data:processed/X_test.csv', encoding='utf-8')

In [20]:
print(X_test.shape)
y_test.head()

(14852, 33)


36434    1
58903    0
49997    0
51498    0
37112    1
Name: is_canceled, dtype: int64

In [26]:
#y_test.to_csv('../src:data/src:data:processed/y_test.csv', encoding='utf-8')

----